In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pymupdf4llm
import uuid
from arai.utils import Document
from arai.llm_utils import llm_wrapper

In [7]:
path = './IS_example/STORM.pdf'

In [8]:
pages = pymupdf4llm.to_markdown(path, page_chunks=True)

Processing ./IS_example/STORM.pdf...
[                                        ] (0/2[=                                       ] ( 1/2[==                                      ] ( 2/27=[====                                    ] ( 3/27[=====                                   ] ( 4/2=[=======                                 ] ( 5/2[========                                ] ( 6/27=[==========                              ] ( 7/27[===========                             ] ( 8/2=[=============                           ] ( 9/2[==============                          ] (10/27=[================                        ] (11/27[=================                       ] (12/2=[===================                     ] (13/2[====================                    ] (14/27=[======================                  ] (15/27[=======================                 ] (16/2=[=========================               ] (17/2[==========================              ] (18/27=[============================     

In [249]:
documents = [
    Document(
        id=str(uuid.uuid4()), 
        page_content=page['text'], 
        metadata=dict(
            page=page['metadata']['page'],
            source=page['metadata']['file_path']
        )
    ) 
    for page in pages
]

In [250]:
page_1 = documents[0]

background = llm_wrapper(prompt=f"{page_1.page_content}\n\nRead the text above and summarize it.")

In [3]:
from arai.agents.inputagent import InputAgent
from arai.agents.organizer import OrganizerAgent
from arai.agents.jargon_detector import JargonDetector
from arai.memories import EventBus

event_bus = EventBus()
input_agent = InputAgent()
organizer_agent = OrganizerAgent()
jargon_translator = JargonDetector()

In [4]:
input_agent.start_conversation(user_input="S.O.S. is an abbreviation for Save Our Souls.")

InputAgent: Processing input: S.O.S. is an abbreviation for Save Our Souls.
OrganizerAgent: Routing request...
OrganizerAgent: parsing
JargonDetector: Extracting jargons, abbreviations, acronyms from INPUT...
JargonDetector: parsing


In [5]:
event_bus.get_latest_event()

{'user_input': 'S.O.S. is an abbreviation for Save Our Souls.',
 'jargons': {'jargons': [{'jargon': 'S.O.S.'}]},
 'source': 'JargonDetector',
 'timestamp': 1734830378,
 'event_type': 'editing_needed'}

In [6]:
event_bus.get_events()

[{'user_input': 'S.O.S. is an abbreviation for Save Our Souls.',
  'source': 'InputAgent',
  'timestamp': 1734830376,
  'event_type': 'input_received'},
 {'user_input': 'S.O.S. is an abbreviation for Save Our Souls.',
  'reason': 'INPUT contains jargons, abbreviations, or acronyms',
  'source': 'OrganizerAgent',
  'timestamp': 1734830377,
  'event_type': 'jargon_needed'},
 {'user_input': 'S.O.S. is an abbreviation for Save Our Souls.',
  'jargons': {'jargons': [{'jargon': 'S.O.S.'}]},
  'source': 'JargonDetector',
  'timestamp': 1734830378,
  'event_type': 'editing_needed'}]

In [7]:
event_bus.clear_event_bus()